In [5]:
import cv2
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import tf2onnx
import pickle

In [6]:
dataset = tf.keras.utils.image_dataset_from_directory("Dataset/", batch_size=512, image_size=(120,120), shuffle=False)

Found 512 files belonging to 2 classes.


2023-11-22 17:17:59.572564: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/asif/Documents/GitHub/ApML/Lab3/.venv/lib/python3.7/site-packages/cv2/../../lib64:
2023-11-22 17:17:59.572664: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-22 17:17:59.572705: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu-asif): /proc/driver/nvidia/version does not exist
2023-11-22 17:17:59.573691: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow wi

In [7]:
x = np.asarray(list(dataset.as_numpy_iterator())[0][0])/255
y = np.asarray(tf.one_hot(list(dataset.as_numpy_iterator())[0][1],depth=2))

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.7,shuffle=True)

In [9]:
M1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(48, 6, strides=(4, 4), input_shape=(120, 120, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(64, 5, padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(64, 3, padding='same'),
    tf.keras.layers.Conv2D(64, 3, padding='same'),
    tf.keras.layers.Conv2D(64, 3, padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.Dense(2, 'sigmoid')
])

In [10]:
M1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
M1.fit(x_train, y_train, epochs=10)

Epoch 1/10
12/12 [==============================] - 12s 278ms/step - loss: 0.4965 - accuracy: 0.7709
Epoch 2/10
12/12 [==============================] - 3s 262ms/step - loss: 0.3180 - accuracy: 0.9134
Epoch 3/10
12/12 [==============================] - 3s 207ms/step - loss: 0.0117 - accuracy: 0.9972
Epoch 4/10
12/12 [==============================] - 3s 217ms/step - loss: 0.0076 - accuracy: 0.9972
Epoch 5/10
12/12 [==============================] - 3s 231ms/step - loss: 0.0463 - accuracy: 0.9860
Epoch 6/10
12/12 [==============================] - 3s 259ms/step - loss: 0.0235 - accuracy: 0.9916
Epoch 7/10
12/12 [==============================] - 3s 222ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 8/10
12/12 [==============================] - 3s 222ms/step - loss: 9.5340e-04 - accuracy: 1.0000
Epoch 9/10
12/12 [==============================] - 3s 239ms/step - loss: 2.6161e-04 - accuracy: 1.0000
Epoch 10/10
12/12 [==============================] - 3s 253ms/step - loss: 2.1458e-04 - acc

In [12]:
M1.evaluate(x_test, y_test)

5/5 [==============================] - 1s 70ms/step - loss: 5.6802e-05 - accuracy: 1.0000


[5.680219692294486e-05, 1.0]

In [13]:
with open("dset.pickle", "wb") as f:
    pickle.dump((x, y), f)

In [14]:
spec = (tf.TensorSpec((None, 120, 120, 3), tf.float32, name="input"),)
output_path = "M1.onnx"
model_proto, _ = tf2onnx.convert.from_keras(M1, input_signature=spec, opset=13, output_path=output_path)

2023-11-22 17:19:34.624232: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-11-22 17:19:34.886374: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
